In [1]:
import pandas as pd
import numpy as np

In [10]:
columns = [
    "NU_INSCRICAO",
    "NU_IDADE",
    "TP_SEXO",
    "TP_ST_CONCLUSAO",
    "TP_ANO_CONCLUIU",
    "TP_ESCOLA",
    "IN_TREINEIRO",
    "TP_PRESENCA_CN",
    "TP_PRESENCA_CH",
    "TP_PRESENCA_LC",
    "TP_PRESENCA_MT",
    "CO_PROVA_CN",
    "CO_PROVA_CH",
    "CO_PROVA_LC",
    "CO_PROVA_MT",
    "NU_NOTA_CN",
    "NU_NOTA_CH",
    "NU_NOTA_LC",
    "NU_NOTA_MT",
    "TX_RESPOSTAS_CN",
    "TX_RESPOSTAS_CH",
    "TX_RESPOSTAS_LC",
    "TX_RESPOSTAS_MT",
    "TP_LINGUA",
    "TX_GABARITO_CN",
    "TX_GABARITO_CH",
    "TX_GABARITO_LC",
    "TX_GABARITO_MT"]

df = pd.read_parquet("MICRODADOS_ENEM_2019_SAOPAULO.parquet", columns=columns)
df_comp = pd.read_parquet("MICRODADOS_ENEM_2019_SAOPAULO.parquet")

In [ ]:
df_comp.columns.shape

In [3]:
qtdCandidatos = df.shape[0]
qtdAusentes = df[(df["TP_PRESENCA_CN"]==0) & (df["TP_PRESENCA_CH"]==0) & (df["TP_PRESENCA_LC"]==0) & (df["TP_PRESENCA_MT"]==0)].shape[0]
qtdFaltou =  df[(df["TP_PRESENCA_CN"]==0) | (df["TP_PRESENCA_CH"]==0) | (df["TP_PRESENCA_LC"]==0) | (df["TP_PRESENCA_MT"]==0)].shape[0]
qtdFaltEliminados = df[(df["TP_PRESENCA_CN"]!=1) | (df["TP_PRESENCA_CH"]!=1) | (df["TP_PRESENCA_LC"]!=1) | (df["TP_PRESENCA_MT"]!=1)].shape[0]

print(f'Quantidade total de candidatos: {qtdCandidatos}')
print(f'Quantidade de candidatos que faltaram em todas as provas: {qtdAusentes} ({round(qtdAusentes*100/qtdCandidatos, 2)})%')
print(f'Quantidade de candidatos que faltaram em uma das provas: {qtdFaltou} ({round(qtdFaltou*100/qtdCandidatos,2)})%')
print(f'Quantidade de candidatos que faltaram ou foram eliminados em uma das provas: {qtdFaltEliminados} ({round(qtdFaltEliminados*100/qtdCandidatos,2)})%')

Quantidade total de candidatos: 229837
Quantidade de candidatos que faltaram em todas as provas: 55340 (24.08)%
Quantidade de candidatos que faltaram em uma das provas: 67047 (29.17)%
Quantidade de candidatos que faltaram ou foram eliminados em uma das provas: 67178 (29.23)%


In [9]:
df[df['IN_TREINEIRO'] == 1].shape[0]/df.shape[0] * 100

9.69469667634019

In [4]:
dfFiltered = df[(df["TP_PRESENCA_CN"]==1) | (df["TP_PRESENCA_CH"]==1) | (df["TP_PRESENCA_LC"]==1) | (df["TP_PRESENCA_MT"]==1)]

dfFiltered[["TX_RESPOSTAS_CN",
    "TX_RESPOSTAS_CH",
    "TX_RESPOSTAS_LC",
    "TX_RESPOSTAS_MT",
    "TP_LINGUA",
    "TX_GABARITO_CN",
    "TX_GABARITO_CH",
    "TX_GABARITO_LC",
    "TX_GABARITO_MT"]].head()

In [8]:
arrayResp = {"CN":[],"CH":[], "MT":[], "LC":[]}

for subject in arrayResp.keys():

    n = 46 if subject != "LC" else 51

    columnsNames = [subject + str(i) for i in range(1,n)]
    dictNames =  dict(enumerate(columnsNames,1))

    dfSubject = dfFiltered[["TX_RESPOSTAS_" + subject, "TX_GABARITO_" + subject]].dropna()

    dfRespostas = dfSubject["TX_RESPOSTAS_" + subject].str.split("",0,expand=True).drop(0, axis="columns", inplace=False).drop(n, axis="columns", inplace=False)        
    dfRespostas.rename(columns=dictNames, inplace=True)

    dfGabarito = dfSubject["TX_GABARITO_" + subject].str.split("",0,expand=True).drop(0, axis="columns", inplace=False).drop(n, axis="columns", inplace=False)
    dfGabarito.rename(columns=dictNames, inplace=True)

    dfCN = pd.DataFrame()

    for column in columnsNames:
        conditions = [dfRespostas[column] == dfGabarito[column], pd.isna(dfRespostas[column])]
        choices = [1, np.nan]

        dfRespostas[column] = np.select(conditions, choices, default=0)
    
    arrayResp[subject] = dfRespostas


In [29]:
arrayProvas = {"CN","CH", "MT", "LC"}

for prova in arrayProvas:
    print(f"Prova: {prova}")
    print(", ".join([str(item) for item in dfFiltered["CO_PROVA_" + prova].unique()]) + "\n")

Prova: CH
509.0, 510.0, 507.0, 508.0, 524.0, nan, 520.0

Prova: LC
514.0, 513.0, 511.0, 512.0, 525.0, nan, 521.0

Prova: MT
518.0, 516.0, 515.0, 517.0, nan, 526.0, 522.0

Prova: CN
505.0, 504.0, 503.0, 506.0, nan, 523.0, 519.0

